# Importing packages and dataset

In [70]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension()
import hvplot.pandas
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [52]:
df = pd.read_csv('owid-co2-data.csv')
df.head(2)
df.shape

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,co2_including_luc_per_capita,co2_including_luc_per_gdp,co2_including_luc_per_unit_energy,co2_per_capita,co2_per_gdp,co2_per_unit_energy,coal_co2,coal_co2_per_capita,consumption_co2,consumption_co2_per_capita,consumption_co2_per_gdp,cumulative_cement_co2,cumulative_co2,cumulative_co2_including_luc,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_luc_co2,cumulative_oil_co2,cumulative_other_co2,energy_per_capita,energy_per_gdp,flaring_co2,flaring_co2_per_capita,gas_co2,gas_co2_per_capita,ghg_excluding_lucf_per_capita,ghg_per_capita,land_use_change_co2,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,other_co2_per_capita,other_industry_co2,primary_energy_consumption,share_global_cement_co2,share_global_co2,share_global_co2_including_luc,share_global_coal_co2,share_global_cumulative_cement_co2,share_global_cumulative_co2,share_global_cumulative_co2_including_luc,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_luc_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_luc_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1850.0,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.931,0.781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.121,NaN,NaN,NaN,NaN,0.121,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851.0,AFG,3769828.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.968,0.787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.119,NaN,NaN,NaN,NaN,0.118,NaN,NaN,NaN,NaN,NaN,NaN


(13728, 74)

# Some data preprocessing

In [53]:
df = df.fillna(0)
df['gdp_per_capita'] = np.where(df['population'] != 0, df['gdp']/df['population'], 0)

In [71]:
# Make dataframe pipeline interactive
idf = df.interactive()

In [72]:
idf

# (1) CO2 emission over time by continent

In [56]:
# Define panel widgets
year_slider = pn.widgets.IntSlider(name = 'Year slider', start = 1750, end = 2021, step = 5, value = 1850)
year_slider

IntSlider(end=2021, name='Year slider', start=1750, step=5, value=1850)

In [57]:
# Radio buttons for CO2 measures
yaxis_co2 = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['co2', 'co2_per_capita',],
    button_type = 'success'
)

In [61]:
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antartica']

co2_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents))
    ]
    .groupby(['country', 'year'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [62]:
co2_plot = co2_pipeline.hvplot(x='year', 
                               by='country', 
                               y=yaxis_co2, 
                               line_width=2, 
                               title="CO2 emission by continent")
co2_plot

# (2) Table - CO2 over time by continent

In [63]:
co2_table = co2_pipeline.pipe(pn.widgets.Tabulator, 
                              pagination='remote', 
                              page_size=10, 
                              sizing_mode='stretch_width')
co2_table

# (3) CO2 vs GDP scatterplot

In [65]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
        (~ (idf.country.isin(continents)))
    ]
    .groupby(['country', 'year', 'gdp_per_capita'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop = True)
)

In [68]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp_per_capita', 
                                                                by='country', 
                                                                y='co2', 
                                                                size=80, 
                                                                kind='scatter',
                                                               alpha=0.7,
                                                               legend=False,
                                                               height=500,
                                                               width=500)

# (4) Bar chart with CO2 sources by continent

In [69]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name = 'Y axis',
    options = ['coal_co2', 'oil_co2', 'gas_co2'],
    button_type = 'success'
)

continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antartica']

co2_source_bar_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')
    .reset_index(drop=True)
)

In [73]:
co2_source_barplot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                    x='country', 
                                                    y=yaxis_co2_source, 
                                                    title='CO2 source by continent')

# Creating dashborad

In [81]:
template = pn.template.FastListTemplate(
    title = 'World CO2 emission dashboard',
    sidebar = [pn.pane.Markdown("# CO2 emissions and climate change"),
               pn.pane.Markdown("#### Carbon dioxyde emissions are the primary driver of global climate change ..."),
               pn.pane.Markdown("## Settings"),
               year_slider],
    main = [pn.Row(pn.Column(yaxis_co2,
                             co2_plot.panel(width=700), margin=(0,25)),
                  co2_table.panel(width=500)),
            pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=600), margin=(0,25)),
                   pn.Column(yaxis_co2_source, co2_source_barplot.panel(width=600)))],
    accent_base_color = '#88d8b0',
    header_background = '#88d8b0'
)

template.show()
# template.servable();